# Quantum Pipeline

Define small feature map (angle encoding, ZZFeatureMap).

Define variational ansatz (1–3 layers).

Measure expectation values → feature vector.

Train classical classifier on these features.

In [1]:
import pennylane as qml
from qiskit.visualization import array_to_latex
import pennylane.numpy as np
from qiskit.circuit.library import ZZFeatureMap, PauliFeatureMap
from qiskit_machine_learning.circuit.library import RawFeatureVector
import matplotlib.pyplot as plt

In [2]:
data = np.load("data/mnist01_pca4.npz")    #from dataprep notebook

X_train, y_train = data["X_train"], data["y_train"]  
X_test, y_test = data["X_test"], data["y_test"]

# x = features/ dimensions --> 4
# y = labels

print("PCA reduced shape xtrain:  ", X_train.shape)
print("PCA reduced shape xtest:  ", X_test.shape)
print("ytrain labels: ", y_train.shape)
print("ytest labels: ",y_test.shape)


PCA reduced shape xtrain:   (12665, 4)
PCA reduced shape xtest:   (2115, 4)
ytrain labels:  (12665,)
ytest labels:  (2115,)


## Quantum Feature Map

Classical data is transformed into quantum states. For the main pipeline, I will be using PennyLane as it is easier to build custom circuits and I can compare feature maps in a single framework. Pennylane also supports Qiskit backeneds for a future plugin to run on IBM devices. 


<br>

This is a quantum map $\phi(\mathbf{x})$ from classical feature vector $\mathbf{x}$ to quantum state $|\Phi(\mathbf{x})\rangle\langle\Phi(\mathbf{x})|$ in the Hilbert space, achieved through applying the parameterized unitary operator $\mathcal{U}_{\Phi(\mathbf{x})}$ to the initial state $|0\rangle^{n}$, n being the num of qubits for encoding. This is done so quantum algorithms can use the data. 

Unitary operations are a building block of Qcircuits, where a matrix U is unitary if $$U^\dagger U = U U^\dagger = I$$ 

Some examples are the Pauli X Gate or Hadamard Gate, or in this case, the Rotation Gate $R_y(\theta)$. 
$$
R_y(\theta) = \begin{bmatrix}\cos(\theta/2) & -\sin(\theta/2) \\ \sin(\theta/2) & \cos(\theta/2)\end{bmatrix}, \quad
R_y^\dagger(\theta) = R_y(-\theta), \quad
R_y(\theta) R_y^\dagger(\theta) = I
$$


### ZZ Map Manual - PennyLane

For a ZZ Feature Map, PennyLane does not already have a built in function so we have to do it manually using Hadamard gates, along wiht RZ and ZZ rotations for each qubit pair. This just makes it simpler to integrate over the course of this project. The Qiskit version of this gate is shown in my quantum testing ground. 

Also, out of all encoding methods I could've picked, Angle Encoding seemed to be the most simple and effective when working with circuits (perfect when I am working on a simple project).

The ZZ feature map is a version of the Pauli feature map, which includes entangling gates (ZZ interactions thorugh controlled-phase gates), done after initial data encoding rotations. This is done to allow feature map to capture any potential correlations between features and then maps the data to a more complex hilbert space. 

There are layers of Hadamard gates, single qubit phase rotations, and 2 qubit controlled phase rotations. 

<br>

Hadamard Gates

Creates equal superposition of either state within a qubit

$$
H = \tfrac{1}{\sqrt{2}}
\begin{pmatrix}
1 & 1 \\[6pt]
1 & -1
\end{pmatrix}
$$

RZ Gates 

Single qubit rotation around Z axis, diagonal gate

$$
R_Z(\phi) 
= \exp\!\left(-i \tfrac{\phi}{2} Z\right) 
= 
\begin{pmatrix}
e^{-i\phi/2} & 0 \\[6pt]
0 & e^{\,i\phi/2}
\end{pmatrix}
$$


CNOT Gates

Flips target qubit if control qubit is 1. 

$$
\text{CNOT} = CX =
\begin{pmatrix}
1 & 0 & 0 & 0 \\[6pt]
0 & 1 & 0 & 0 \\[6pt]
0 & 0 & 0 & 1 \\[6pt]
0 & 0 & 1 & 0
\end{pmatrix}
$$

<br>

In [6]:
sample_vec = X_train[0]      #1st of 12665
print("Example feature vector: ", sample_vec)  


Example feature vector:  [-4.39083685  1.26077612  1.578084    0.85940065]


In [7]:
def zz_fm(x, wires, reps=1):      #(classical data vector, qubits in use, number of layers/repititions)
    n_qubits = len(wires)

    for _ in range(reps):
        for i in wires: 
            qml.Hadamard(wires=i)            #hadamard gates, uniform superposition of states --> gives room for inference in future entanglement 

        for i, wire in enumerate(wires):     #single qubit phase rotations (encodes classical data to qubit phases)
            qml.RZ(2*x[i], wires=wire)       #RZ rotation, angle = 2x[i] 

        for i in range(n_qubits-1):                    #ZZ entangling layer for pairs of neighboring qubits/wires
            qml.CNOT(wires=[wires[i], wires[i+1]])
            qml.RZ(2*x[i]*x[i+1], wires=wires[i+1])    #performs rotation around Z axis by given amount (angle = 2x[i] * x[i+1]) ENCODES PAIRWISE CORRELATIONS BETWEEN FEATURES x[i] and x[i+1]
            qml.CNOT(wires=[wires[i], wires[i+1]])     #uncomputes entanglement (so final effect is ZZ interaction)



dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def feature_map_zz(x):
    zz_fm(x, wires=range(4), reps=1)
    return qml.state()


print(f"ZZFeatureMap state vector: \n {feature_map_zz(sample_vec)} \n") 
print(qml.draw(feature_map_zz, level='device')(sample_vec))    


ZZFeatureMap state vector: 
 [-0.24166348+0.06402159j  0.12856535+0.21440838j  0.24664338+0.04082946j
  0.16880032-0.18440839j -0.02931574-0.24827522j -0.23039082+0.09705706j
  0.24496536+0.04991964j  0.17550227-0.17804199j -0.07578422+0.23823675j
  0.24991017+0.00670117j  0.16690489-0.18612565j -0.06489921-0.24142927j
  0.20609065+0.14151552j  0.07877125-0.23726587j -0.19894346+0.15139848j
  0.01831538+0.24932819j] 

0: ──H──RZ(-8.78)─╭●─────────────╭●─────────────────────────────────┤ ╭State
1: ──H──RZ(2.52)──╰X──RZ(-11.07)─╰X─╭●───────────╭●─────────────────┤ ├State
2: ──H──RZ(3.16)────────────────────╰X──RZ(3.98)─╰X─╭●───────────╭●─┤ ├State
3: ──H──RZ(1.72)────────────────────────────────────╰X──RZ(2.71)─╰X─┤ ╰State


In this circuit, you can see how each qubit has a Hadamard gate applied (without this, qubits starting in |0⟩ and RZ would only add a global phase - unobservable, as RZ works with relative phase differences between |0⟩ and |1⟩). 

Then, a phase rotation of (x) amount around the Z axis to encode the classical features x[i] as a Z axis phase shift of angle 2x[i].  

Finally, we have the entanglement layers that work through each adjacent pair of qubits to perform phase shifts. Applies controlled phase depending on both qubits and encodes pairwaise correlations into the state. 

**Creates a nonlienar feature map in Hilbert space.**

A CNOT sandwich, which is seen within the multiqubit entanglement layer above, allows for conditional application of a targets rotation, depending on the control qubit. So, in the end, we are maintaining the computational values of the qubit but keeping the phase entanglement of the qubits (it would look the same on the 0,1 level but now hidden quantum phase correlations encode the data)